Here I load and parse the input genomes and gene annotations files for pretraining the model.

In [1]:
%reload_ext autoreload
%autoreload 2

# Load package utils
from splicevo.io.utils import load_json

# Dictionary of genomes with paths to files
genome_dict_fn = "/work/aelek/projects/splicing/data/genomes.txt"

genome_dict = load_json(genome_dict_fn)

# Downloaded genomes data
genome_dir = "/work/aelek/projects/splicing/data/genomes"

### Load gene annotations

We have 3 sources of GTF files

In [2]:
from pandas import DataFrame

genome_data = DataFrame.from_dict(genome_dict, orient='index')
genome_data.groupby("source").size().sort_values()

source
mazin        6
ensembl    122
toga       275
dtype: int64

In Ensembl gtf files, the coordinates are 1-based, and both start and end are inclusive. So, for sequence `ACGT`, the record: `chr1 1 3` will correspond to `ACG`.  
Mazin genomes are gtf files based on Ensembl annotations, and Toga annotation gtfs follow the same rules.

However, different files have different records features. We load them here to inspect.

In [3]:
import pandas as pd
from helpers import read_gtf

species = genome_data.query("source == 'mazin'").index[0]
gtf_mazin = read_gtf(genome_dict[species]["gtf"])
gtf_mazin['feature'].unique().tolist()


['transcript', 'exon']

In [4]:
# Select the species with toga annotations
species = genome_data.query("source == 'toga'").index[0]
gtf_toga = read_gtf(genome_dict[species]["gtf"])
gtf_toga['feature'].unique().tolist()

['transcript', 'exon', 'CDS', 'start_codon', 'stop_codon']

In [5]:
# Select the species with toga annotations
species = genome_data.query("source == 'ensembl'").index[0]
gtf_ensembl = read_gtf(genome_dict[species]["gtf"])
gtf_ensembl['feature'].unique().tolist()

['gene',
 'transcript',
 'exon',
 'CDS',
 'stop_codon',
 'start_codon',
 'five_prime_utr',
 'three_prime_utr']

To process gtf files, I define `GTFProcessor` class in splicevo package, and `Transcript` class to parse splice sites coordinates from each transcript. 

In [6]:
from splicevo.io.gene_annotation import GTFProcessor

species = "Homo_sapiens"
gtf_path = genome_dict[species]["gtf"]
test_chromosomes = ["21"]

gtf = GTFProcessor(gtf_path)
gtf_df = gtf.load_gtf(test_chromosomes)
gtf_df

Loading GTF file...
Loaded 36517 GTF records


,chrom,source,feature,start,end,score,strand,frame,gene_id,transcript_id,exon_number
0,21,StringTie,transcript,9548317,9548827,1000,+,.,hum.40401,hum.40401.1,NaN
1,21,StringTie,exon,9548317,9548827,1000,+,.,hum.40401,hum.40401.1,1
2,21,StringTie,transcript,9438205,9454931,1000,+,.,hum.40405,hum.40405.1,NaN
3,21,StringTie,exon,9438205,9439580,1000,+,.,hum.40405,hum.40405.1,1
4,21,StringTie,exon,9439897,9442444,1000,+,.,hum.40405,hum.40405.1,2
...,...,...,...,...,...,...,...,...,...,...,...
36512,21,StringTie,exon,48024926,48025093,1000,-,.,hum.41653,hum.41653.2,3
36513,21,StringTie,transcript,48019175,48024987,1000,+,.,hum.41654,hum.41654.1,NaN
36514,21,StringTie,exon,48019175,48019416,1000,+,.,hum.41654,hum.41654.1,1
36515,21,StringTie,exon,48022191,48022329,1000,+,.,hum.41654,hum.41654.1,2


In [ ]:
gtf_df = gtf.filter_high_confidence(gtf_df)
transcripts = gtf.get_transcripts(gtf_df)
transcripts

In [8]:
transcripts[1].exons

,chrom,source,feature,start,end,score,strand,frame,gene_id,transcript_id,exon_number
1,21,StringTie,exon,9438205,9439580,1000,+,.,hum.40405,hum.40405.1,1
2,21,StringTie,exon,9439897,9442444,1000,+,.,hum.40405,hum.40405.1,2
3,21,StringTie,exon,9442625,9446317,1000,+,.,hum.40405,hum.40405.1,3
4,21,StringTie,exon,9447922,9448040,1000,+,.,hum.40405,hum.40405.1,4
5,21,StringTie,exon,9451875,9451962,1000,+,.,hum.40405,hum.40405.1,5
6,21,StringTie,exon,9452458,9453009,1000,+,.,hum.40405,hum.40405.1,6
7,21,StringTie,exon,9454611,9454931,1000,+,.,hum.40405,hum.40405.1,7


In [9]:
transcripts[1].introns

[(np.int64(9439581), np.int64(9439896)),
 (np.int64(9442445), np.int64(9442624)),
 (np.int64(9446318), np.int64(9447921)),
 (np.int64(9448041), np.int64(9451874)),
 (np.int64(9451963), np.int64(9452457)),
 (np.int64(9453010), np.int64(9454610))]

In [10]:
transcripts[1].splice_donor_sites, transcripts[1].splice_acceptor_sites

({np.int64(9439581),
  np.int64(9442445),
  np.int64(9446318),
  np.int64(9448041),
  np.int64(9451963),
  np.int64(9453010)},
 {np.int64(9439896),
  np.int64(9442624),
  np.int64(9447921),
  np.int64(9451874),
  np.int64(9452457),
  np.int64(9454610)})

This is implemented in `GTFProcessor.process_gtf`.

In [4]:
from splicevo.io.gene_annotation import GTFProcessor

species = "Rattus_norvegicus"
gtf_path = genome_dict[species]["gtf"]
test_chromosomes = ["19"]

gtf = GTFProcessor(gtf_path)
transcripts = gtf.process_gtf(chromosomes=test_chromosomes)

Loading GTF file...
Loaded 67525 GTF records


## Genome

Next, I define a function that will read genomes and load them to OHE numpy array.
Use `CustomGenome` from `grelu` to load genome fasta files.

I realized here that `pyfaidx` does not work with my fasta files compressed with gzip. However, when I decompress them and re-compress using bgzip it works fine (but not with parallel implementation of bgzip, pbgzip!)

In [2]:
from splicevo.io.genome import load_genome, encode_transcript

species = "Rattus_norvegicus"
#fasta_path = genome_dict[species]["fasta"]
fasta_path = "/work/aelek/data/mazin/fasta/Rattus_norvegicus.fasta.bgz"

genome = load_genome(fasta_path)

In [6]:
encode_transcript(transcripts[1], genome)

(tensor([[0, 0, 0,  ..., 1, 0, 1],
         [1, 1, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 1,  ..., 0, 0, 0]], dtype=torch.int8),
 tensor([0, 0, 0,  ..., 0, 0, 0]))